# Pathweigh Analysis

This notebook demonstrates pathway analysis of RNA-seq sequencing data using PathWeigh. It is recommended to clone this repository by using:
```
git clone https://github.com/zurkin1/Yada.git
!pip install -r ../requirements.txt
```
and then run it in a Jupyter notebook.

The input file `./data/input.csv` is located in the data folder.

In [1]:
from udp import *

# We need can configure the following parameter is_rnaseq == True in order to select RNA-seq data
# processing or microarray data processing. 
calc_udp_multi_process(is_rnaseq = True)

Fri Sep  6 10:17:41 2024 Calculate UDP, is_rnaseq: True
..................................................Fri Sep  6 10:23:10 2024 Done.


,17-002,17-006,17-019,17-023,17-026,17-027,17-030,17-034,17-040,17-041,17-042,17-043,17-045,17-047,17-054,17-055,17-057,17-058,17-060,17-061
GeneSym,,,,,,,,,,,,,,,,,,,,
7a5,0.466695,0.638102,0.638102,0.466695,0.638102,0.291047,0.525962,0.466695,0.583448,0.525962,0.583448,0.466695,0.466695,0.638102,0.583448,0.525962,0.406879,0.406879,0.638102,0.466695
a1bg,0.364542,0.900828,0.415207,0.612862,0.516567,0.516567,0.466157,0.516567,0.364542,0.516567,0.773386,0.315032,0.415207,0.612862,0.466157,0.415207,0.900828,0.900828,0.612862,0.900828
a1cf,0.636114,0.458723,0.520014,0.688823,0.579536,0.688823,0.520014,0.579536,0.579536,0.636114,0.520014,0.636114,0.520014,0.520014,0.636114,0.579536,0.579536,0.688823,0.458723,0.688823
a26c3,0.396579,0.455696,0.338548,0.725774,0.514578,0.571994,0.230997,0.678348,0.455696,0.455696,0.725774,0.626874,0.725774,0.514578,0.282937,0.626874,0.725774,0.725774,0.455696,0.678348
a2bp1,0.670267,0.503387,0.443577,0.383688,0.503387,0.670267,0.443577,0.670267,0.503387,0.383688,0.561822,0.617753,0.670267,0.670267,0.383688,0.443577,0.383688,0.443577,0.383688,0.503387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyg11a,0.759685,0.677734,0.477553,0.720547,0.677734,0.423564,0.795015,0.423564,0.477553,0.477553,0.795015,0.477553,0.530768,0.795015,0.423564,0.423564,0.530768,0.795015,0.530768,0.631521
zyg11b,0.629372,0.019689,0.735585,0.900154,0.657508,0.220309,0.900154,0.271472,0.015741,0.154089,0.271472,0.196753,0.900154,0.900154,0.570649,0.386490,0.570649,0.710713,0.759203,0.570649
zyx,0.771535,0.292669,0.835107,0.902406,0.513309,0.717255,0.902406,0.636005,0.531031,0.395290,0.902406,0.031803,0.422692,0.036859,0.185028,0.902406,0.301318,0.134973,0.098462,0.706885


## Calculate Activity For all Samples and For All Pathways.

In [5]:
from udp import *
from activity import *


udp = pd.read_csv('./data/output_udp.csv', index_col=0, encoding="utf-8")
activity_obj = path_activity(udp, is_rnaseq = True)
activity = activity_obj.calc_activity_consistency_multi_process() #Output is saved to output_activity.csv.

activity_obj.activity.head()

Fri Sep  6 10:26:03 2024 Init activity object
Fri Sep  6 10:26:04 2024 Calculate activity and consistency...
.Fri Sep  6 10:26:15 2024 Done.


sampleID,17-002,17-006,17-019,17-023,17-026,17-027,17-030,17-034,17-040,17-041,17-042,17-043,17-045,17-047,17-054,17-055,17-057,17-058,17-060,17-061
path_name,,,,,,,,,,,,,,,,,,,,
1- and 2-Methylnaphthalene degradation(Kegg),0.632876,0.660917,0.562899,0.444796,0.473169,0.448846,0.471721,0.656867,0.623339,0.588166,0.56648,0.632876,0.482305,0.588166,0.542506,0.595516,0.558011,0.563706,0.645239,0.576818
1-4-Dichlorobenzene degradation(Kegg),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3-Chloroacrylic acid degradation(Kegg),0.779011,0.823286,0.796039,0.753257,0.68915,0.728662,0.829729,0.810999,0.725207,0.724963,0.833427,0.796039,0.74945,0.724963,0.675395,0.759734,0.678203,0.819587,0.691467,0.75756
Acute myeloid leukemia(Kegg),0.359899,0.365481,0.398714,0.345536,0.344096,0.316362,0.377824,0.383984,0.460113,0.342276,0.392314,0.2915,0.337403,0.355758,0.366264,0.423215,0.361883,0.257353,0.37749,0.33395
Adherens junction(Kegg),0.234253,0.244784,0.290832,0.229037,0.178929,0.216386,0.281799,0.307647,0.27055,0.283174,0.310374,0.308393,0.296344,0.256935,0.273138,0.304501,0.248184,0.198864,0.216739,0.298867


## Draw a Heatmap For Path Activity.

The following UI demonstrations were done using the following libraries versions:

- plotly 5.9.0
- ipywidgets 7.6.5 (plotly currently cannot work with ipywidgets version 8). It might be possible to run other combinations.

In [8]:
import plotly.express as px
import plotly.graph_objects as go


sample_num = 0
path_id = 708708 #1-and 2-Methylnaphthalene degradation(Kegg).

df = activity_obj.activity.transpose()

#for col in df:
#    if df[col].mean() < 0.5:
#        df.drop(col, inplace=True, axis=1)

#fig = px.imshow(df, text_auto = True, aspect = 'auto', width = 1000, height = 800)
fig = go.FigureWidget(go.Heatmap(x=df.columns, y=df.index, z=df.to_numpy()))
fig.update_layout(width = 1000, height = 1000)

#Create our callback function.
hmap = fig.data[0]
fig.layout.hovermode = 'closest'

def update_point(trace, points, selector):
    path_id = points.xs[0]
    sample_num = points.ys[0]
    activity_obj.graphparser(path_id=path_id, sample_num=sample_num)
    #xml_result = activity_obj.xmlparser(path_id=path_id, sample_num=sample_num)


hmap.on_click(update_point)

fig

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '31c40e18-7806-4c3e-bdd7-1fc8b950b8de',
              'x': array(['1- and 2-Methylnaphthalene degradation(Kegg)',
                          '1-4-Dichlorobenzene degradation(Kegg)',
                          '3-Chloroacrylic acid degradation(Kegg)', ...,
                          'wnt signaling pathway(BioCarta)',
                          'y branching of actin filaments(BioCarta)',
                          'yaci and bcma stimulation of b cell immune responses(BioCarta)'],
                         dtype=object),
              'y': array(['17-002', '17-006', '17-019', '17-023', '17-026', '17-027', '17-030',
                          '17-034', '17-040', '17-041', '17-042', '17-043', '17-045', '17-047',
                          '17-054', '17-055', '17-057', '17-058', '17-060', '17-061'],
                         dtype=object),
              'z': array([[0.6328758897939857, 0.0, 0.7790109375671856, ..., 0.058610049181779

## Download a KGML File to Import to Cytoscape.


- Configure sample_num.
- Configure path_id for analysis and export. Use the following table:
https://github.com/zurkin1/Pathweigh/blob/master/README.md.

In [10]:
import bs4
from activity import *


sample_num = 0
path_id = 708708

xml_result = activity_obj.xmlparser(path_id=path_id, sample_num=sample_num)
print(bs4.BeautifulSoup(xml_result).prettify())

Create Kegg XML for path: 708708, sample: 0


AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

## Draw a Pathway Activity Map.

We can select a specific sample and a specific pathway and draw the activity map.

In [6]:
sample_num = 0
path_id = 708708

activity_obj.graphparser(path_id, sample_num)

## Troubleshooting
- It was reported on problems running in Pycharm due to multiprocessing issues. There are no problems running on the command line or in Colab.